# **SynthDNA Pipeline**

This notebook contains the code used to generate and filter DNA polymerase sequences I tested. There are three main sections: sequence generation, filtering, and fidelity scoring.

In [ ]:
# Installing dependencies

!pip install -U transformers
!pip install accelerate
!pip install -U datasets
!pip install scikit-learn
!pip install peft
!pip install -U bitsandbytes
!apt-get update
!apt-get install -y clustalo
!pip install biopython
!pip install deepspeed
!apt-get -qq install cd-hit
!pip install bioemu

## **Part 1: Generating Sequences**

In [ ]:
# Setting device to GPU

import pandas as pd
import numpy as np
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

GPU is available


In [ ]:
# loading model for sequence generation

# make sure to add tokenizer file to runtime

from peft import AutoPeftModelForCausalLM
from tokenizers import Tokenizer
from transformers import BitsAndBytesConfig
import torch.nn.functional as F
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import deepspeed

model = AutoPeftModelForCausalLM.from_pretrained("divyanb/SynthDNA-Model", trust_remote_code=True,
        torch_dtype=torch.float16,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        ))

tokenizer = Tokenizer.from_file('tokenizer.json')

ds_inference_config = {
    "replace_with_kernel_inject": True,
    "dtype": torch.float16,
}

model = deepspeed.init_inference(model, config=ds_inference_config, mp_size=1)

model.to(device)

In [ ]:
batch_size = 16
num_sequences = 5000
generated_sequences = []

tokenized_sequence = tokenizer.encode('2SKKINLWSTLGVQRTKQYRLDEKRYGFGELIRLVAPLVQNEIYYEADYKHKKPDYEEALIARNSIPGDGRLVIYGIVMGPKIKVGKAALKKAVAVHPGIAKYEHLPRTIQEYIALKEPPIEYNALKQIVEKVIRVAEEVDGHKLITELVRAQTEKAIESWDRRVIELGRTIVKGEEDIVAYRKKTVFFGRKYFGEYELELLGPLKSNIYKVFELAKKKIEESEGGPITAYLGDTDIYLVKFGFKEELEKWVLEIYKRGWATVSEACEKCYWRAKAYGYYGYFSNALLKIAKQRYDLLIKEIPDQTEKMKTKIKQREELLHGLLSPIFGPIDKCFKHGVQPAIDYNKCGELNLTDPSVNHTIIISPYLARFDLYVINEWLGKEPEKVFGGTYSERLRRQYEEESPKNPAVENREYAKRLLFWEVLN')

input_tensor = torch.tensor([tokenized_sequence.ids]).to(device)

for i in range(0, num_sequences, batch_size):
    current_batch_size = min(batch_size, num_sequences - i)

    with torch.no_grad():
        output = model.generate(
            input_tensor,
            max_length=800,
            pad_token_id=tokenizer.encode('<|pad|>').ids[0],
            do_sample=True,
            top_p=0.5, #for N term constant I used 0.9
            temperature=0.5, #for N term constant I used 1.0
            num_return_sequences=current_batch_size
        )

    as_lists = lambda batch: [batch[i, ...].detach().cpu().numpy().tolist() for i in range(batch.shape[0])]
    sequences = tokenizer.decode_batch(as_lists(output))

    if len(sequences) > 0:
        sequences = [x.replace('2', '') for x in sequences]
        sequences = [x.replace('1', '') for x in sequences]

    generated_sequences.extend(sequences)
    print(len(generated_sequences))

In [ ]:
import csv

with open('batchX.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    for string in generated_sequences:
        writer.writerow([string])

print("CSV file created successfully.")

## **Part 2: Filtering Sequences**

In [ ]:
# Read CSV file with generated sequences

import csv

with open('batchX.csv', 'r', newline='') as file:
    reader = csv.reader(file)
    sequences = [row[0] for row in reader]

print(len(sequences))

In [ ]:
# Define functions for filtering

import subprocess
import re
from torch.nn.functional import softmax

# Function to run Clustal Omega for two sequences
def run_clustal_omega(seq1, seq2, output_file):
    with open("input_sequences.fasta", "w") as f:
        f.write(f">Sequence1\n{seq1}\n")
        f.write(f">Sequence2\n{seq2}\n")

    clustal_cmd = [
        "clustalo",
        "-i", "input_sequences.fasta",
        "-o", output_file,
        "--force",
        "--outfmt", "clu"
    ]

    subprocess.run(clustal_cmd)

    with open(output_file, "r") as aligned_file:
        alignment = aligned_file.read()
    return alignment

# Function to parse the Clustal Omega alignment file
def parse_alignment_file(alignment_file):
    sequences = {}
    with open(alignment_file, 'r') as file:
        current_sequence = ""
        for line in file:
            # Skip headers or other non-sequence lines
            if line.startswith(" ") or line.startswith("CLUSTAL") or not line.strip():
                continue

            # Split the line into sequence name and sequence fragment
            parts = line.split()
            if len(parts) < 2:
                continue
            sequence_name, sequence_fragment = parts[0], parts[1]

            # Append the fragment to the corresponding sequence
            if sequence_name not in sequences:
                sequences[sequence_name] = ""
            sequences[sequence_name] += sequence_fragment

    return sequences

# Function to check if specific residues are conserved, allowing for a set of acceptable residues
def check_conserved_residues_with_gaps(reference_seq, generated_seq, residues_to_check):
    ref_position = 0
    conserved = True

    for i in range(len(reference_seq)):  # Loop through the aligned sequence
        if reference_seq[i] != '-':  # Only increment the reference position if it's not a gap
            ref_position += 1

        if ref_position in residues_to_check:  # Check only the positions of interest
            ref_residue = reference_seq[i]
            gen_residue = generated_seq[i]

            allowed_residues = residues_to_check[ref_position]

            # Check if the generated residue is within the allowed set
            if gen_residue not in allowed_residues:
                conserved = False

    return conserved

residues_to_check = {
    141: 'D',  # Aspartate at position 141
    142: ['I', 'V'],  # Isoleucine or Valine at position 142
    143: 'E',  # Glutamate at position 143
    210: 'N',  # Asparagine at position 210
    214: 'F',  # Phenylalanine at position 214
    215: 'D',  # Aspartate at position 215
    274: ['S', 'T'], #Serine or Threonine at position 274
    275: 'L',  # Leucine at position 275
    278: ['A', 'V', 'L', 'I', 'M', 'F', 'W', 'P', 'Y'],  #Hydrophobic at position 278
    287: 'K',  # Lysine at position 287
    311: 'Y',  # Tyrosine at position 311
    315: 'D',  # Aspartate at position 315
    387: 'G',  # Glycine at position 387
    390: 'V',  # Valine at position 390
    405: 'D',  # Aspartate at position 405
    408: ['S', 'A'], # Serine or Aspartate at position 408
    409: 'L',  # Leucine at position 409
    410: 'Y',  # Tyrosine at position 410
    411: 'P',  # Proline at position 411
    412: 'S',  # Serine at position 412
    413: 'I',  # Isoleucine at position 413
    414: 'I',  # Isoleucine at position 414
    484: 'Q',  # Glutamine at position 484
    488: 'K',  # Lysine at position 488
    492: 'N',  # Asparagine at position 492
    493: 'S',  # Serine at position 493
    495: 'Y',  # Tyrosine at position 495
    496: 'G',  # Glycine at position 496
    518: 'G',  # Glycine at position 518
    541: 'D',  # Aspartate at position 541
    542: 'T',  # Threonine at position 542
    543: 'D',  # Aspartate at position 543
    581: 'E',  # Glutamate at position 581
    592: 'K',  # Lysine at position 592
    593: 'K',  # Lysine at position 593
    595: 'Y',  # Tyrosine at position 595
    596: 'A',  # Alanine at position 596
    608: 'G',  # Glycine at position 608
    610: 'E',  # Glutamate at position 610
}

# Example sequences
sequence_2 = "MILDVDYITEEGKPVIRLFKKENGKFKIEHDRTFRPYIYALLRDDSKIEEVKKITGERHGKIVRIVDVEKVEKKFLGKPITVWKLYLEHPQDVPTIREKVREHPAVVDIFEYDIPFAKRYLIDKGLIPMEGEEELKILAFDIETLYHEGEEFGKGPIIMISYADENEAKVITWKNIDLPYVEVVSSEREMIKRFLRIIREKDPDIIVTYNGDSFDFPYLAKRAEKLGIKLTIGRDGSEPKMQRIGDMTAVEVKGRIHFDLYHVITRTINLPTYTLEAVYEAIFGKPKEKVYADEIAKAWESGENLERVAKYSMEDAKATYELGKEFLPMEIQLSRLVGQPLWDVSRSSTGNLVEWFLLRKAYERNEVAPNKPSEEEYQRRLRESYTGGFVKEPEKGLWENIVYLDFRALYPSIIITHNVSPDTLNLEGCKNYDIAPQVGHKFCKDIPGFIPSLLGHLLEERQKIKTKMKETQDPIEKILLDYRQKAIKLLANSFYGYYGYAKARWYCKECAESVTAWGRKYIELVWKELEEKFGFKVLYIDTDGLYATIPGGESEEIKKKALEFVKYINSKLPGLLELEYEGFYKRGFFVTKKRYAVIDEEGKVITRGLEIVRRDWSEIAKETQARVLETILKHGDVEEAVRIVKEVIQKLANYEIPPEKLAIYEQITRPLHEYKAIGPHVAVAKKLAAKGVKIKPGMVIGYIVLRGDGPISNRAILAEEYDPKKHKYDAEYYIENQVLPAVLRILEGFGYRKEDLRYQKTRQVGLTSWLNIKKS"


In [ ]:
# Define more functions and run the filtering process

def is_degenerate(sequence):
    # Define the k-mer sizes and the maximum allowed repetitions
    k_mer_rules = {
        6: 2,  # 6-mer cannot repeat more than 2 times consecutively
        5: 3,
        4: 3,  # 4-mer cannot repeat more than 3 times consecutively
        3: 6,  # 3-mer cannot repeat more than 6 times consecutively
        2: 8,   # 2-mer cannot repeat more than 8 times consecutively
        7: 2,
        8: 2
    }

    for k, max_repeats in k_mer_rules.items():
        # Check each k-mer in the sequence
        for i in range(len(sequence) - k + 1):
            k_mer = sequence[i:i + k]
            repeat_count = sequence.count(k_mer * max_repeats)
            if repeat_count > 0:
                return True

    return False


def contains_non_canonical_amino_acids(sequence):
    # Canonical amino acids
    canonical_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
    # Check if any amino acid in the sequence is non-canonical
    return any(aa not in canonical_amino_acids for aa in sequence)

def uncompleted(sequence):
    if (sequence[0] == 'M'):
      return False
    else:
      return True


# Full pipeline: check conserved residues, filter degenerate, and non-canonical sequences
for seq in sequences[:]:  # Using a copy of the list for safe removal
    if is_degenerate(seq) or contains_non_canonical_amino_acids(seq) or uncompleted(seq):
        sequences.remove(seq)

for seq in sequences[:]:
    sequence_1 = seq
    alignment_result = run_clustal_omega(sequence_1, sequence_2, "aligned_sequences.aln")

    alignment_file = "aligned_sequences.aln"
    aligned_sequences = parse_alignment_file(alignment_file)

    reference_seq = aligned_sequences['Sequence2']
    generated_seq = aligned_sequences['Sequence1']

    residues_conserved = check_conserved_residues_with_gaps(reference_seq, generated_seq, residues_to_check)

    # Corrected comparison and list removal
    if residues_conserved == False:
        sequences.remove(seq)

1059


In [ ]:
# Filter sequences further based on sequence perplexity scores

for seq in sequences[:]:
    #perplexity filtering
    tokenized_sequence = tokenizer.encode(f'1{seq}2')
    input_tensor = torch.tensor([tokenized_sequence.ids]).to('cuda')
    output = model(input_tensor)
    logits = output.logits

    # Convert logits to probabilities
    probabilities = softmax(logits, dim=-1)

    # Calculate cross-entropy
    true_token_ids = input_tensor[:, 1:]  # Shifted true token IDs for labels
    log_probs = torch.log(probabilities)

    # Fix dimension mismatch by unsqueezing true_token_ids
    true_token_ids = true_token_ids.unsqueeze(-1)  # Now true_token_ids shape is (batch_size, seq_length-1, 1)

    # Gather log_probs based on the true_token_ids
    selected_log_probs = torch.gather(log_probs[:, :-1, :], dim=-1, index=true_token_ids)

    # Calculate cross-entropy and perplexity
    cross_entropy = -torch.mean(selected_log_probs)
    perplexity = torch.exp(cross_entropy)

    if (perplexity.item() - 2.2079501152038574) > 0.7:
      sequences.remove(seq)

print(len(sequences))

In [ ]:
# Filter out sequences with 80% sequence identity to each other

input_fasta = "input_sequences.fasta"
with open(input_fasta, "w") as f:
    for i, seq in enumerate(sequences):
        f.write(f">seq{i}\n{seq}\n")

!cd-hit -i input_sequences.fasta -o clustered_sequences.fasta -c 0.8 -n 5 -d 0

def read_fasta(filename):
    filtered = []
    with open(filename, 'r') as f:
        seq = ""
        for line in f:
            if line.startswith(">"):
                if seq:
                    filtered.append(seq)
                    seq = ""
            else:
                seq += line.strip()
        if seq:
            filtered.append(seq)
    return filtered

sequences = read_fasta("clustered_sequences.fasta")

In [ ]:
# Load progen2-base for checking log likelihood

!git clone https://github.com/salesforce/progen
%cd progen/progen2

# checkpoint
model='progen2-base'
!wget -P checkpoints/progen2-base https://storage.googleapis.com/sfr-progen-research/checkpoints/progen2-base.tar.gz
!tar -xvf checkpoints/progen2-base/progen2-base.tar.gz -C checkpoints/progen2-base/

!pip3 install --upgrade pip setuptools
!pip3 install -r requirements.txt
%cd ..
%cd ..

In [ ]:
# Load ESM model for checking log likelihood

!pip install fair-esm

import torch
import esm
from transformers import AutoModelForMaskedLM

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

# Load the ESM-2 3B model from Hugging Face
model_name = "facebook/esm2_t36_3B_UR50D"
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)
model.eval()  # disables dropout for deterministic results
model.to(device)

In [ ]:
# Compute log likelihood of sequences with ESM

def compute_pseudolikelihoods(sequences):
    # Store pseudolikelihoods
    pseudolikelihoods = []

    with torch.no_grad():
        for sequence in sequences:
            # Prepare the sequence for the model
            batch_labels, batch_strs, batch_tokens = batch_converter([("protein", sequence)])
            batch_tokens = batch_tokens.to(device)

            # Calculate log-likelihoods
            logits = model(batch_tokens)["logits"]
            log_probs = logits.log_softmax(dim=-1)

            # Extract pseudolikelihood for the sequence
            sequence_length = batch_tokens.shape[1] - 2  # Exclude special tokens ([CLS], [EOS])
            batch_tokens_no_special = batch_tokens[:, 1:-1]

            # Masked positions' probabilities
            masked_probs = log_probs[0, torch.arange(1, sequence_length + 1), batch_tokens_no_special[0]]
            pseudolikelihood = (masked_probs.sum().item())/len(sequence)

            pseudolikelihoods.append(pseudolikelihood)

    return pseudolikelihoods

likelihoodsESM = compute_pseudolikelihoods(sequences)

In [ ]:
# Compute log likelihoods with ProGen2

import json

with open('sequences.json', 'w') as f:
    json.dump(sequences, f)

%cd progen/progen2
!python3 likelihood.py --context-file sequences.json

with open('likelihoods.json', 'r') as f:
    likelihoodsProGen = json.load(f)

In [ ]:
# Filter down to the top 100 sequences with the highest log likelihood scores

import numpy as np

likelihoodsProGen = np.array(likelihoodsProGen)  # Replace with your Progen2 likelihoods
likelihoodsESM = np.array(likelihoodsESM)    # Replace with your ESM-2 likelihoods

average_likelihoods = (likelihoodsProGen + likelihoodsESM) / 2

closest_indices = np.argsort(np.abs(average_likelihoods))[:100]

sequences = [sequences[i] for i in closest_indices]

In [ ]:
# Creates a FASTA file of the sequences to input to NetSolP

def write_fasta(sequences, output_file, headers=None):

    if headers is None:
        headers = [f"seq{i+1}" for i in range(len(sequences))]
    if len(sequences) != len(headers):
        raise ValueError("The number of headers must match the number of sequences.")

    with open(output_file, "w") as fasta_file:
        for header, sequence in zip(headers, sequences):
            fasta_file.write(f">{header}\n")
            fasta_file.write(f"{sequence}\n")
    print(f"FASTA file written to {output_file}")

output_path = "protein_sequences.fasta"

write_fasta(sequences, output_path)

In [ ]:
# Arrange results from NetSolP and filter down to the top 20

import pandas as pd
import numpy as np

sequences = pd.read_csv('NetSolP.csv')
fasta = sequences['fasta'].to_list()
solubility = sequences['predicted_solubility'].to_list()
usability = sequences['predicted_usability'].to_list()

combined = list(zip(fasta, solubility, usability))

sorted_combined = sorted(combined, key=lambda x: x[2])

sorted_fasta, sorted_solubility, sorted_usability = zip(*sorted_combined)

sorted_fasta = list(sorted_fasta)
sorted_solubility = list(sorted_solubility)
sorted_usability = list(sorted_usability)

selected_sequences = sorted_fasta[80:]

In [ ]:
print(sorted_solubility[80:100])
print(sorted_usability[80:100])

In [ ]:
# Create CSV file with the sequences to input into Boltz

with open('batchX_selected.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each string as a new row
    for string in selected_sequences:
        writer.writerow([string])

print("CSV file created successfully.")

In [ ]:
# Create a FASTA file for each sequence in a directory called "sequences"

import os

# Path to your CSV file
csv_file_path = "selected_sequences2.csv"

# Create the 'sequences' directory if it doesn't exist
output_dir = "sequences"
os.makedirs(output_dir, exist_ok=True)

# Read the CSV file and create FASTA files
with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.reader(csv_file)

    # Process each row (single column: sequence)
    for row_number, row in enumerate(csv_reader, start=1):
        if not row:
            continue  # Skip empty rows

        sequence = row[0]
        fasta_file_path = os.path.join(output_dir, f"sequence{row_number}.fasta")

        # Write to the FASTA file
        with open(fasta_file_path, "w") as fasta_file:
            fasta_file.write(f">A|protein\n{sequence}")

print(f"FASTA files have been created in the '{output_dir}' directory.")

In [ ]:
# Extracting predictions file

import zipfile

zip_file_path = 'predictions.zip'
extract_dir = 'predictions'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")

In [ ]:
# Arrange sequences from lowest to highest confidence scores

parent_dir = 'predictions/boltz_results_sequences/predictions'

# List to store the confidence scores
confidence_scores = []
directory_names = []

# Walk through the directories in the parent directory
for dir_name in os.listdir(parent_dir):
    dir_path = os.path.join(parent_dir, dir_name)

    # Check if the directory name starts with 'sequence'
    if os.path.isdir(dir_path) and dir_name.startswith('sequence'):
        # Look for the JSON file in the directory
        for file_name in os.listdir(dir_path):
            if file_name.endswith('.json'):
                json_file_path = os.path.join(dir_path, file_name)

                # Read the JSON file and extract the confidence score
                with open(json_file_path, 'r') as f:
                    data = json.load(f)
                    # Append the confidence score and directory name
                    confidence_scores.append(data.get('confidence_score'))
                    directory_names.append(dir_name)

# Combine the lists and sort by confidence scores
sorted_pairs = sorted(zip(confidence_scores, directory_names))

# Unzip the sorted pairs back into separate lists
sorted_confidence_scores, sorted_directory_names = zip(*sorted_pairs)

# Convert them back to lists (if needed)
sorted_confidence_scores = list(sorted_confidence_scores)
sorted_directory_names = list(sorted_directory_names)

# Print the sorted confidence scores and corresponding directory names
print("Sorted Confidence Scores:", sorted_confidence_scores)
print("Sorted Directory Names:", sorted_directory_names)

## **Part 3: Fidelity Scoring**

In [ ]:
# Using BioEmu to obtain a simulation of the DNA polymerase (input sequence and replace X with sequence number)

from bioemu.sample import main as sample
sample(sequence='', num_samples=1000, output_dir='./sequenceX')

In [ ]:
# Enter positions of aspartate residues in designed polymerase (D405 and D541 in WT Pfu)

!apt-get install -y clustalo

def run_clustal_omega(seq1, seq2, output_file):
    with open("input_sequences.fasta", "w") as f:
        f.write(f">Sequence1\n{seq1}\n")
        f.write(f">Sequence2\n{seq2}\n")

    clustal_cmd = [
        "clustalo",
        "-i", "input_sequences.fasta",
        "-o", output_file,
        "--force",
        "--outfmt", "clu"
    ]

    subprocess.run(clustal_cmd)

    with open(output_file, "r") as aligned_file:
        alignment = aligned_file.read()
    return alignment

# Function to parse the Clustal Omega alignment file
def parse_alignment_file(alignment_file):
    sequences = {}
    with open(alignment_file, 'r') as file:
        current_sequence = ""
        for line in file:
            # Skip headers or other non-sequence lines
            if line.startswith(" ") or line.startswith("CLUSTAL") or not line.strip():
                continue

            # Split the line into sequence name and sequence fragment
            parts = line.split()
            if len(parts) < 2:
                continue
            sequence_name, sequence_fragment = parts[0], parts[1]

            # Append the fragment to the corresponding sequence
            if sequence_name not in sequences:
                sequences[sequence_name] = ""
            sequences[sequence_name] += sequence_fragment

    return sequences

def check_for_aspartate(reference_seq, generated_seq, residues_to_check):
    ref_position = 0
    gaps = 0
    conserved = True

    for i in range(len(reference_seq)):  # Loop through the aligned sequence
        if reference_seq[i] != '-':  # Only increment the reference position if it's not a gap
            ref_position += 1

        if generated_seq[i] == '-':
            gaps += 1

        if ref_position in residues_to_check:  # Check only the positions of interest
            return i-gaps+1

sequence_2 = "MILDVDYITEEGKPVIRLFKKENGKFKIEHDRTFRPYIYALLRDDSKIEEVKKITGERHGKIVRIVDVEKVEKKFLGKPITVWKLYLEHPQDVPTIREKVREHPAVVDIFEYDIPFAKRYLIDKGLIPMEGEEELKILAFDIETLYHEGEEFGKGPIIMISYADENEAKVITWKNIDLPYVEVVSSEREMIKRFLRIIREKDPDIIVTYNGDSFDFPYLAKRAEKLGIKLTIGRDGSEPKMQRIGDMTAVEVKGRIHFDLYHVITRTINLPTYTLEAVYEAIFGKPKEKVYADEIAKAWESGENLERVAKYSMEDAKATYELGKEFLPMEIQLSRLVGQPLWDVSRSSTGNLVEWFLLRKAYERNEVAPNKPSEEEYQRRLRESYTGGFVKEPEKGLWENIVYLDFRALYPSIIITHNVSPDTLNLEGCKNYDIAPQVGHKFCKDIPGFIPSLLGHLLEERQKIKTKMKETQDPIEKILLDYRQKAIKLLANSFYGYYGYAKARWYCKECAESVTAWGRKYIELVWKELEEKFGFKVLYIDTDGLYATIPGGESEEIKKKALEFVKYINSKLPGLLELEYEGFYKRGFFVTKKRYAVIDEEGKVITRGLEIVRRDWSEIAKETQARVLETILKHGDVEEAVRIVKEVIQKLANYEIPPEKLAIYEQITRPLHEYKAIGPHVAVAKKLAAKGVKIKPGMVIGYIVLRGDGPISNRAILAEEYDPKKHKYDAEYYIENQVLPAVLRILEGFGYRKEDLRYQKTRQVGLTSWLNIKKS"

In [ ]:
sequence = "MILDVDYITEEGKPVIRLFKKENGKFKIEHDRTFRPYIYALLRDDSKIEEVKKITGERHGKIVRIVDVEKVEKKFLGKPITVWKLYLEHPQDVPTIREKVREHPAVVDIFEYDIPFAKRYLIDKGLIPMEGEEELKILAFDIETLYHEGEEFGKGPIIMISYADENEAKVITWKNIDLPYVEVVSSEREMIKRFLRIIREKDPDIIVTYNGDSFDFPYLAKRAEKLGIKLTIGRDGSEPKMQRIGDMTAVEVKGRIHFDLYHVITRTINLPTYTLEAVYEAIFGKPKEKVYADEIAKAWESGENLERVAKYSMEDAKATYELGKEFLPMEIQLSRLVGQPLWDVSRSSTGQLVEWFLLRKAYERNELVPNKPSEKEYAQRKGSYAGGYVKEPLRGLHENIALFDFRSLYPSIIVTHNIGPDTLNCKCCKDEEYEVPGFQFMFCKKRKGFIPAAIEEIIERRVRVKELMKGASDPLEKQLLDARQFALKIIANSFYGYLGWFRARWYKQECAASVTAFGRYYIGQVIKAAEEEGFKVLYGDTDSLFVTVGKNSPEEAIKFLERINRELPGVMELELEGFYPRGIFVSKKRYAVISDDGKITVKGLEAVRRDWSVIAKEVQTKVLEIILREGDPKKAAELVRKVIKDLREGRVPIDKLIIYTQLTKDPNKYEATAPHVRAAKKLMEMGYKVGKGDKIGYVIVKGSGKVSERAYPYEKVSSKDYDVDYYIKNQIIPAVMRVLEAFGVKEEELIAGGKQIDLFRFG"

alignment_result = run_clustal_omega(sequence, sequence_2, "aligned_sequences.aln")

alignment_file = "aligned_sequences.aln"
aligned_sequences = parse_alignment_file(alignment_file)

reference_seq = aligned_sequences['Sequence2']
generated_seq = aligned_sequences['Sequence1']

aspartate_residue1 = check_for_aspartate(reference_seq, generated_seq, {405: 'D'})
aspartate_residue2 = check_for_aspartate(reference_seq, generated_seq, {541: 'D'})

In [ ]:
aspartate_residue1 = 2
aspartate_residue2 = 3

In [ ]:
!pip install MDAnalysis

import MDAnalysis as mda
from MDAnalysis.analysis import rms, distances
from MDAnalysis.analysis.rms import RMSD, RMSF
import numpy as np
import matplotlib.pyplot as plt

u = mda.Universe("topology1.pdb", "samples1.xtc")

alignment = RMSD(u, u, select="backbone", ref_frame=0)
alignment.run()
rmsd_times = alignment.rmsd[:,1]
rmsd_values = alignment.rmsd[:,2]

plt.figure()
plt.plot(rmsd_times, rmsd_values)
plt.xlabel("Time (ps)")
plt.ylabel("Backbone RMSD (Å)")
plt.title("Backbone RMSD vs. Time")
plt.show()

prot = u.select_atoms("protein and backbone")
rmsf_calc = RMSF(prot).run()

ix_to_rmsf = dict(zip(prot.atoms, rmsf_calc.rmsf))

rmsf_per_residue = []
resnums = []

for res in prot.residues:
    res_rmsfs = [ix_to_rmsf[atom] for atom in res.atoms if atom in ix_to_rmsf]
    if res_rmsfs:
        rmsf_per_residue.append(np.mean(res_rmsfs))
        resnums.append(res.resid)

plt.figure()
plt.plot(resnums, rmsf_per_residue)
plt.xlabel("Residue Number")
plt.ylabel("Backbone RMSF (Å)")
plt.title("Backbone RMSF per Residue")
plt.show()


sel1 = u.select_atoms(f"resid {aspartate_residue1} and name CA")
sel2 = u.select_atoms(f"resid {aspartate_residue2} and name CA")
distances_over_time = []
times = []

for ts in u.trajectory:
    pos1 = sel1.positions
    pos2 = sel2.positions
    dist = np.linalg.norm(pos1 - pos2)
    distances_over_time.append(dist)
    times.append(ts.time)

plt.figure()
plt.plot(times, distances_over_time)
plt.xlabel("Time (ps)")
plt.ylabel("Distance (Å)")
plt.title("Catalytic CA–CA Distance vs. Time")
plt.show()

print(f"Average backbone RMSD: {np.mean(rmsd_values):.2f} Å ± {np.std(rmsd_values):.2f} Å")
catalytic_residues = [aspartate_residue1, aspartate_residue2]
active_site_rmsfs = [r for r, n in zip(rmsf_per_residue, resnums) if n in catalytic_residues]
print(f"Max RMSF in active-site residues: {max(active_site_rmsfs):.2f} Å")
print(f"Average catalytic distance: {np.mean(distances_over_time):.2f} Å")

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, pca
import numpy as np
import matplotlib.pyplot as plt

selection = "protein and name CA"

atoms = u.select_atoms(selection)

aligner = align.AlignTraj(u, u, select=selection, in_memory=True)
aligner.run()

pca_analysis = pca.PCA(u, select=selection, align=False)
pca_analysis.run()

projections = pca_analysis.transform(atoms, n_components=2)

print("Variance explained by PC1 and PC2:")
print(pca_analysis.results.variance[:2])